trying to do kalman filtering just from simon's paper

In [5]:
%%capture clear
%pip install numpy
%pip install matplotlib

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x_axis = np.arange(-2,10,0.01)

# Kalman Filtering

* so we need to choose some noise say of covar 0.01 for both observation and target
* lets also choose to do a system with two variables x1 and x2
* ~~ lets also choose the dynamics of the system to be just a linear system say 0.5,0.5 ~~
* timesteps of one second
* we're also making all matricies and noise time invariant / constants to save some time

In [64]:
# just gives our random vars for v and n sampled form standard norm
#(then divided to make smaller)
covar_q = [0.01,0.1 ]# this could vary in time in this case it doesn't
covar_r = [0.01, 0.01] # same with this one
def v_func()->list[float]:
    return (np.linalg.matmul(np.random.randn(2),covar_q))
def n_func()->list[float]:
    return (np.linalg.matmul(np.random.randn(2),covar_r))

def update_target(F : list[list[float]],
       X : list[list[float]]
       ) -> list[float]:
    return (np.linalg.matmul(F,X[-1])+v_func())

def update_observation(H : list[list[float]],
                       X : list[list[float]]
                       ) -> list[float]:
    return np.matmul(H,X[-1])+n_func()


time_step = np.arange(0,10.1,1)
# define the dynamics of the system
f_matrix = [[1,1],
            [0,1]]
# define the target matrix and its inistial positions
target = [[0.5,0.5]]

# to begin with lets have observations just be double the target
# and have that be our observation
h_matrix = [[2,0],
            [0,2]]
observations = []

observations += [update_observation(h_matrix,target)]
print(observations)


for time in time_step:
    if not time:
        continue
    target += [update_target(f_matrix,target)]

print(np.array(target))

[array([0.98938826, 0.98938826])]
[[0.5        0.5       ]
 [1.0830575  0.5830575 ]
 [1.71074196 0.62768446]
 [2.27137138 0.56062942]
 [2.99051321 0.71914184]
 [3.69901378 0.70850057]
 [4.47422528 0.7752115 ]
 [5.10120698 0.6269817 ]
 [5.73586228 0.6346553 ]
 [6.50002804 0.76416575]
 [7.2942907  0.79426266]]


ok so going off of this equation

![images](images/Screenshot%20from%202024-10-14%2013-07-51.png)

**btw P is a covar value**

to the best of my knowledge for covar_k-1|k-1 we would want to use a list covar_k|k and sample the value from the previous time step to get the value we want 

In [108]:

# this will be a mean x1,x2 and a covar value for each value for
# each timestep
prediction = []
m_k_k1 = []

covar_k_k1 = []
# needs to be initialised idk what to though
# only somehting that need to be initialised at k=0
# for the maths to work should be initialised at k=0
# I think anyway

# giving the actual covar vals probably too low if anythin
covar_k_k = [[0.01,0.01]]
# just give the actual val at time 0 idk if, idk if this is the
# right thing to do though
m_k_k = [[0.5,0.5]] 

f_matrix = np.array(f_matrix) # so i can use .T
h_matrix = np.array(h_matrix) # so i can use .T


# k1 is used to mean k-1
# the underscore inbetween the two vars should be taken to mean
# the pipe operator so given in stats
target += [update_target(f_matrix, target)]
observations += [update_observation(h_matrix,target)]

m_k_k1 += [np.matmul(f_matrix,m_k_k[-1])]
temp = np.linalg.matmul(f_matrix,covar_k_k[-1])
covar_k_k1 += [covar_q + np.linalg.matmul(temp,f_matrix.T)]

temp = np.linalg.matmul(h_matrix,covar_k_k1[-1])
sk = np.linalg.matmul(temp,h_matrix.T)+covar_r
temp = np.linalg.matmul(covar_k_k1,h_matrix.T)
try:
    sk_inv = np.linalg.inv(sk)
    kalman_gain = np.linalg.matmul(temp,sk_inv)
except:
    kalman_gain = temp/sk

temp = observations[-1] - np.linalg.matmul(h_matrix,m_k_k1[-1])
m_k_k += [m_k_k1[-1] + np.linalg.matmul(kalman_gain,temp)]
temp = np.linalg.matmul(kalman_gain,h_matrix)
covar_k_k += [covar_k_k1 - np.linalg.matmul(temp[0],covar_k_k1[-1])]



In [ ]:

for time in time_step:
    # basically target at timestep 0 is already defined so skip
    # in this case but no other case
    if time:
        target += [update_target()]
    observations = []
    